## Pairwise Distances!

timewise vs. space
timewise vs. pairwise diff

In [2]:
import allel
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle as pkl
import json
import umap

# for the maps
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

### time and pairwise distances

In [3]:
vcf_path = 'data/variants.vcf.gz'
metadata = pd.read_csv("data/fixed.csv")

# this reads the vcf file into a callset
callset = allel.read_vcf(vcf_path)

# this makes the genotype allele!
gt = allel.GenotypeArray(callset['calldata/GT'])
variants = pd.DataFrame({
    'CHROM': callset['variants/CHROM'],
    'POS': callset['variants/POS'],
    'ID': callset['variants/ID'],
    'REF': callset['variants/REF'],
    'ALT': [a[0] if isinstance(a, (list, tuple)) and len(a) > 0 else None for a in callset['variants/ALT']]
})

gt = gt.to_n_alt()

/Users/skylarwalters/miniconda3/envs/cs2370/lib/python3.13/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)


Set up distance matrix

In [17]:
dist_df = pd.DataFrame(gt.T)
samples = metadata["Sample_Name"]
ages = [int(age[:4]) for age in metadata["Age"]]
dist_df.rows = samples

/var/folders/g2/5glzdcws7bbg271xmljmgdqh0000gn/T/ipykernel_27068/182130564.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dist_df.rows = samples


In [10]:
# Thank you aley
n = dist_df.shape[0]
dist = np.zeros((n, n))

G = dist_df.values.astype(float)
n = G.shape[0]

dist = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        mask = ~np.isnan(G[i]) & ~np.isnan(G[j])
        d = np.sum(G[i,mask] != G[j,mask])
        dist[i,j] = dist[j,i] = d

dist_df = pd.DataFrame(dist, index=ages, columns=ages)
dist_df

,1092,1098,1150,1090,1444,1521,1700,1500,1250,1090,...,1057,850,650,675,480,1275,938,644,944,1085
1092,0.0,91.0,110.0,78.0,169.0,173.0,9.0,141.0,100.0,63.0,...,34.0,153.0,95.0,108.0,140.0,44.0,121.0,36.0,42.0,184.0
1098,91.0,0.0,185.0,159.0,210.0,204.0,100.0,206.0,189.0,148.0,...,103.0,168.0,176.0,167.0,199.0,103.0,154.0,101.0,121.0,193.0
1150,110.0,185.0,0.0,78.0,143.0,177.0,109.0,109.0,54.0,77.0,...,122.0,171.0,83.0,118.0,100.0,136.0,151.0,142.0,132.0,166.0
1090,78.0,159.0,78.0,0.0,161.0,179.0,79.0,109.0,62.0,73.0,...,92.0,161.0,79.0,108.0,96.0,106.0,157.0,112.0,108.0,170.0
1444,169.0,210.0,143.0,161.0,0.0,224.0,168.0,154.0,153.0,150.0,...,169.0,172.0,156.0,167.0,157.0,185.0,198.0,187.0,173.0,179.0
1521,173.0,204.0,177.0,179.0,224.0,0.0,172.0,204.0,189.0,174.0,...,181.0,218.0,184.0,201.0,197.0,189.0,206.0,183.0,201.0,203.0
1700,9.0,100.0,109.0,79.0,168.0,172.0,0.0,134.0,97.0,64.0,...,43.0,154.0,98.0,115.0,135.0,49.0,118.0,43.0,47.0,177.0
1500,141.0,206.0,109.0,109.0,154.0,204.0,134.0,0.0,103.0,124.0,...,147.0,212.0,118.0,153.0,87.0,153.0,170.0,171.0,155.0,181.0
1250,100.0,189.0,54.0,62.0,153.0,189.0,97.0,103.0,0.0,67.0,...,112.0,175.0,89.0,110.0,88.0,128.0,153.0,134.0,116.0,170.0
1090,63.0,148.0,77.0,73.0,150.0,174.0,64.0,124.0,67.0,0.0,...,81.0,156.0,88.0,105.0,101.0,97.0,132.0,97.0,87.0,169.0


In [16]:

pairs = []

# make rray of ages
ages_array = np.array(dist_df.index).astype(float)
print(ages_array)
n = len(ages_array)

# iterate thru ipper triangle
for i in range(n):
    for j in range(i+1, n):
        age_diff = abs(ages_array[j] - ages_array[i])
        
        
        dist_val = dist_df.iloc[i, j]
        
        # add to pair dict
        pairs.append({
            "age_diff": age_diff,
            "pairwise_diff": dist_val
        })

pair_df = pd.DataFrame(pairs)
pair_df.to_csv('ages_dists.csv')
pair_df

[1092. 1098. 1150. 1090. 1444. 1521. 1700. 1500. 1250. 1090. 1100. 1124.
 1321. 1057.  850.  650.  675.  480. 1275.  938.  644.  944. 1085.]


,age_diff,pairwise_diff
0,6.0,91.0
1,58.0,110.0
2,2.0,78.0
3,352.0,169.0
4,429.0,173.0
...,...,...
248,6.0,121.0
249,147.0,153.0
250,300.0,72.0
251,441.0,182.0


In [ ]:
# Thank you aley
n = dist_df.shape[0]
dist = np.zeros((n, n))

G = dist_df.values.astype(float)
n = G.shape[0]

dist = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        mask = ~np.isnan(G[i]) & ~np.isnan(G[j])
        d = np.sum(G[i,mask] != G[j,mask])
        dist[i,j] = dist[j,i] = d

dist_df = pd.DataFrame(dist, index=ages, columns=ages)
dist_df

latlong = tuple(df.La)
